# More graceful way of stopping loops running in a multiprocess

This uses multiprocessing.Event()

It is really really confusing , but see:
    
https://mail.python.org/pipermail/python-list/2014-June/673646.html

https://mail.python.org/pipermail/python-list/2014-June/673682.html

In [13]:
import multiprocessing
import asyncio
import time

async def test_a(ev):
    while not ev.is_set():
        print('1')
        # time.sleep(1)
        await asyncio.sleep(1)
        
async def test_b(ev):
    while not ev.is_set():
        print('2')
        # time.sleep(1)
        await asyncio.sleep(1)
        
async def stop_it(ev, loop):
    while not ev.is_set():
        await asyncio.sleep(0)

    print('i am here!')
    loop.call_soon_threadsafe(loop.stop)
    
        
        


class testrun(multiprocessing.Process):
    def __init__(self):
        super(testrun, self).__init__()
        self.ev = multiprocessing.Event()
        
        
    def run(self):
        
        
        loop=asyncio.new_event_loop()
        # asyncio.set_event_loop(loop)
        
        loop.create_task(test_a(self.ev))
        loop.create_task(test_b(self.ev))
        loop.create_task(stop_it(self.ev, loop))
        #loop.run_until_complete(tasks)
        loop.run_forever()

        loop.close()
        
        print('loop is stopped!')
        
    
    def stop_it(self):
        self.ev.set()

        

In [14]:
t=testrun()

In [15]:
t.start()

1
2
1
2
1
2
i am here!
loop is stopped!


In [16]:
t.stop_it()

# An event loop is already running in the jupyter notebook!

This causes the following error

In [1]:
import asyncio
import time

async def myTask():
    time.sleep(1)
    print("Processing Task")

async def myTaskGenerator():
    for i in range(5):
        asyncio.ensure_future(myTask())

loop = asyncio.get_event_loop()
loop.run_until_complete(myTaskGenerator())
print("Completed All Tasks")
loop.close()

RuntimeError: This event loop is already running

Processing Task
Processing Task
Processing Task
Processing Task
Processing Task


In [3]:
import asyncio

async def do_io():
    print('io start')
    await asyncio.sleep(5)
    print('io end')

async def do_other_things():
    print('doing other things')

loop = asyncio.new_event_loop()

loop.run_until_complete(do_io())
loop.run_until_complete(do_other_things())

loop.close()

RuntimeError: Cannot run the event loop while another loop is running

In [4]:
import asyncio

async def an_async_fn():
    await asyncio.sleep(1)
    
loop = asyncio.get_event_loop()

loop.run_until_complete(an_async_fn())

RuntimeError: This event loop is already running